The code includes several improvements and advanced features that make it more versatile and suitable for complex analyses. However, some elements could be streamlined for better efficiency and scalability.
- Modular Structure: Separate the code into distinct modules for data generation, calculations, and visualization. This will improve maintainability and scalability.
- Error Handling: Ensure comprehensive error handling across all functions.
- Code Documentation: Add detailed docstrings for each function to improve readability and usability.
- Optimization: Optimize data processing functions to handle large datasets more efficiently.
- 
Functions and Capabilities:
- calculate_percent_rank_inc: Computes incremental percentile ranks for a given data series.
- calculate_percentile_rank_surface: Generates a percentile rank surface for the given underlying asset and date.
- create_vol_surface: Creates a volatility surface for the given date and underlying asset.
- ensure_numerical: Ensures all DataFrame values are numerical.
- style_df: Styles a DataFrame with color gradients for better visualization.
- calculate_z_scores: Calculates z-scores for a series of observations.
- plot_surface: Displays different types of surfaces (Level, Percentile, Z-score) based on user inputs.
- get_vol_moneyness: Generates random volatility moneyness data.
- get_vol_delta: Generates random volatility delta data.
- generate_table: Combines moneyness and delta data in a structured DataFrame.
- df_to_nested_dict: Converts a DataFrame to a nested dictionary.
- sanity_check_surface: Checks the sanity of calculated surfaces against original data.
- calculate_percentile_rank_surface: Similar functionality as in the original code but includes spread calculation between two underlying assets.
- calculate_zscore_surface: Similar functionality as in the original code but includes spread calculation.
- create_vol_surface: Creates a volatility surface for the spread between two underlying assets.
- plot_surface: Similar functionality but includes spread calculations.
- format_and_adjust_column_names: Formats and adjusts column names for better readability.
- BNPP_colors: Sets up a color scheme for styling.

gpt prompt: Merge the strengths of both scripts into a cohesive and scalable solution. Here’s a step-by-step guide to achieve this:


### vol_data.py

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import matplotlib.colors as mcolors
import pickle

# Data Generation Functions
def get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, mon): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for mon in moneyness_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'moneyness'])
    return df

def get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, delta): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for delta in delta_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'delta'])
    return df


### Debugiing

In [16]:
def run_all_checks():
    # Define your test cases here
    pass  # Implement as needed

# Define a debug function
def debug_function(function, **kwargs):
    try:
        result = function(**kwargs)
        print(f"Function {function.__name__} executed successfully.")
        return result, None
    except Exception as e:
        print(f"Function {function.__name__} failed with error: {e}")
        return None, str(e)

# Run debug tests
def run_all_debug_tests(nested_dict):
    test_results = []

    # Test cases with different function and parameters
    test_cases = [
        (create_vol_surface, {"nested_dict": nested_dict, "date": "2024-05-28", "udl": "JP_NKY", "moneyness_levels": [80, 90, 100]}),
        (calculate_percentile_rank_surface, {"nested_dict": nested_dict, "udl": "JP_NKY", "date": "2024-05-28", "moneyness_levels": [80, 90, 100], "start_date": "2024-01-01", "end_date": "2024-05-28"}),
        (calculate_zscore_surface, {"nested_dict": nested_dict, "udl": "JP_NKY", "date": "2024-05-28", "moneyness_levels": [80, 90, 100], "start_date": "2024-01-01", "end_date": "2024-05-28"})
    ]

    for function, params in test_cases:
        result, error = debug_function(function, **params)
        test_results.append({
            "function": function.__name__,
            "params": params,
            "result": "Success" if result is not None else "Failure",
            "error": error
        })

    return test_results

# Format debug report
def format_debug_report(test_results):
    report = "Daily Debug Report:\n\n"
    for result in test_results:
        report += f"Function: {result['function']}\n"
        report += f"Params: {result['params']}\n"
        report += f"Result: {result['result']}\n"
        if result['error']:
            report += f"Error: {result['error']}\n"
        report += "\n"
    return report

### Surface.py

In [17]:
import seaborn as sns
def generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date, surface_data_type='Moneyness'):
    """
    Generate random volatility data for moneyness or delta in a single step and create a structured DataFrame.

    Args:
        udl_list (list): List of underlying assets.
        matu_list (list): List of maturities.
        moneyness_list (list): List of moneyness levels.
        delta_list (list): List of delta levels.
        start_date (str): Start date for the data.
        end_date (str): End date for the data.
        surface_data_type (str): Type of surface data to generate ('Moneyness' or 'Delta').

    Returns:
        DataFrame: DataFrame with combined moneyness or delta data, indexed by date with MultiIndex columns.
    """
    try:
        date_range = pd.date_range(start=start_date, end=end_date, freq='B')
        data = {}

        if surface_data_type == 'Moneyness':
            df_vol_moneyness = get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date)
            for udl in udl_list:
                for matu in matu_list:
                    for mon in moneyness_list:
                        key = (udl, f'IV_{matu}_{mon}')
                        if key in df_vol_moneyness.columns:
                            data[(udl, 'IV', matu, mon)] = df_vol_moneyness[key]
        elif surface_data_type == 'Delta':
            df_vol_delta = get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date)
            for udl in udl_list:
                for matu in matu_list:
                    for delta in delta_list:
                        key = (udl, f'IVFD_{matu}_{delta}')
                        if key in df_vol_delta.columns:
                            data[(udl, 'IVFD', matu, delta)] = df_vol_delta[key]

        df = pd.DataFrame(data, index=date_range)
        df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'param', 'matu', 'value'])
        df.index.name = 'Date'
        df.ffill(inplace=True)

        return df
    except Exception as e:
        print(f"An error occurred during table generation: {e}")
        return pd.DataFrame()

def df_to_nested_dict(df):
    nested_dict = {}
    for date, row in df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        nested_dict[date_str] = {}
        for col, val in row.items():
            udl, param, matu, value = col
            nested_dict[date_str].setdefault(udl, {}).setdefault(param, {}).setdefault(matu, {})[value] = val
    return nested_dict

def calculate_percent_rank_inc(data, start_date=None, end_date=None):
    if isinstance(data, pd.DataFrame):
        if data.shape[1] != 1:
            raise ValueError("DataFrame input should have exactly one column.")
        series = data.iloc[:, 0]
    elif isinstance(data, dict):
        flat_data = {pd.to_datetime(date): value for date, nested_dict in data.items() for key, value in nested_dict.items()}
        series = pd.Series(flat_data)
    elif isinstance(data, pd.Series):
        series = data
    else:
        raise TypeError("Input data should be a pandas Series, DataFrame, or dictionary.")
    
    if not isinstance(series.index, pd.DatetimeIndex):
        series.index = pd.to_datetime(series.index)
    
    if start_date or end_date:
        start_date = pd.to_datetime(start_date) if start_date else series.index.min()
        end_date = pd.to_datetime(end_date) if end_date else series.index.max()
        series = series.loc[start_date:end_date]
    
    if series.empty:
        return pd.Series([], index=series.index)
    
    ranks = series.rank(method='min').apply(lambda x: (x - 1) / (len(series) - 1))
    return ranks

def calculate_z_scores(series):
    return (series - series.mean()) / series.std(ddof=0)

def calculate_percentile_rank_surface(nested_dict, udl1, udl2, date, levels, surface_data_type, start_date=None, end_date=None):
    try:
        date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        if surface_data_type == 'Moneyness':
            data_type_key = 'IV'
        else:
            data_type_key = 'IVFD'

        percentile_rank_surface = pd.DataFrame(index=levels, columns=nested_dict[date_str][udl1][data_type_key].keys())

        for matu in percentile_rank_surface.columns:
            for level in levels:
                try:
                    values = {
                        pd.to_datetime(past_date): nested_dict[past_date][udl1][data_type_key][matu][level] - nested_dict[past_date][udl2][data_type_key][matu][level]
                        for past_date in nested_dict
                        if (udl1 in nested_dict[past_date] and data_type_key in nested_dict[past_date][udl1]
                            and matu in nested_dict[past_date][udl1][data_type_key] and level in nested_dict[past_date][udl1][data_type_key][matu]
                            and udl2 in nested_dict[past_date] and data_type_key in nested_dict[past_date][udl2]
                            and matu in nested_dict[past_date][udl2][data_type_key] and level in nested_dict[past_date][udl2][data_type_key][matu])
                    }
                    if values:
                        series = pd.Series(values)
                        percentile_series = calculate_percent_rank_inc(series, start_date, end_date)
                        percentile_rank_surface.at[level, matu] = percentile_series.get(pd.to_datetime(date_str), np.nan)
                    else:
                        percentile_rank_surface.at[level, matu] = np.nan
                except KeyError:
                    percentile_rank_surface.at[level, matu] = np.nan

        return percentile_rank_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_percentile_rank_surface: {e}")
        return pd.DataFrame()

def calculate_zscore_surface(nested_dict, udl1, udl2, date, levels, surface_data_type, start_date=None, end_date=None):
    try:
        date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        if surface_data_type == 'Moneyness':
            data_type_key = 'IV'
        else:
            data_type_key = 'IVFD'

        zscore_surface = pd.DataFrame(index=levels, columns=nested_dict[date_str][udl1][data_type_key].keys())

        for matu in zscore_surface.columns:
            for level in levels:
                try:
                    values = {
                        pd.to_datetime(past_date): nested_dict[past_date][udl1][data_type_key][matu][level] - nested_dict[past_date][udl2][data_type_key][matu][level]
                        for past_date in nested_dict
                        if (udl1 in nested_dict[past_date] and data_type_key in nested_dict[past_date][udl1]
                            and matu in nested_dict[past_date][udl1][data_type_key] and level in nested_dict[past_date][udl1][data_type_key][matu]
                            and udl2 in nested_dict[past_date] and data_type_key in nested_dict[past_date][udl2]
                            and matu in nested_dict[past_date][udl2][data_type_key] and level in nested_dict[past_date][udl2][data_type_key][matu])
                    }
                    if values:
                        series = pd.Series(values)
                        mean = series.mean()
                        std = series.std()
                        current_value = nested_dict[date_str][udl1][data_type_key][matu][level] - nested_dict[date_str][udl2][data_type_key][matu][level]
                        z_score = (current_value - mean) / std if std != 0 else np.nan
                        zscore_surface.at[level, matu] = z_score
                    else:
                        zscore_surface.at[level, matu] = np.nan
                except KeyError:
                    zscore_surface.at[level, matu] = np.nan

        return zscore_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_zscore_surface: {e}")
        return pd.DataFrame()

# Function to create the volatility surface for the spread
def create_vol_surface(nested_dict, date, udl1, udl2, levels, surface_data_type='Moneyness'):
    date_str = date if isinstance(date, str) else date.strftime('%Y-%m-%d')
    vol_surface = pd.DataFrame(index=levels)
    
    if surface_data_type == 'Moneyness':
        data_type_key = 'IV'
    else:
        data_type_key = 'IVFD'
    
    for matu, data in nested_dict[date_str][udl1][data_type_key].items():
        vol_surface[matu] = [data.get(level, np.nan) - nested_dict[date_str][udl2][data_type_key][matu].get(level, np.nan) for level in levels]
    
    vol_surface = vol_surface.T
    vol_surface.columns = [f'{level}' for level in vol_surface.columns]
    vol_surface.index.name = 'Maturity'
    vol_surface = vol_surface.map(lambda x: round(x, 2) if pd.notna(x) else np.nan)
    return vol_surface

### style.py

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors

BNPP_colors = [
    (0, 124/256, 177/256), # Blue
    (112/256, 194/256, 122/256), # Green
    (238/256, 48/256, 46/256) # Red
]

BNPP_colors_float = [[c/256 for c in color] for color in BNPP_colors]

GR_cmap = mcolors.LinearSegmentedColormap.from_list("GR_cmap", [BNPP_colors[1], BNPP_colors[2]], N=256)

def format_and_adjust_column_names(df):
    formatted_columns = []
    for col in df.columns:
        try:
            float_col = float(col)
            if float_col.is_integer():
                formatted_columns.append(f"{int(float_col)}")
            else:
                formatted_columns.append(f"{float_col:.1f}")
        except ValueError:
            formatted_columns.append(str(col))
    df.columns = formatted_columns
    df.index = [str(idx) for idx in df.index]
    return df

def style_df(df, caption):
    df = ensure_numerical(df)
    if df.isnull().all().all():
        print("The DataFrame contains only NaN values.")
        return df.style.set_caption(caption)
    df = format_and_adjust_column_names(df)
    
    cm = sns.light_palette("green", as_cmap=True)
    
    try:
        # Check for all-NaN slices and handle them
        df_styled = df.style.map(lambda x: 'background-color: #000000' if pd.isna(x) else '')
        df_styled = df_styled.background_gradient(cmap=GR_cmap).format("{:.1f}").set_table_styles([
            {'selector': 'th', 'props': [('min-width', '90px'), ('max-width', '90px'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('text-align', 'center')]}
        ]).set_properties(**{'text-align': 'center'})
        df_styled = df_styled.set_caption(caption).set_table_attributes('style="width:100%; border-collapse:collapse; border: 1px solid black;"')
    except Exception as e:
        print(f"Error during styling: {e}")
        df_styled = df.style.set_caption(caption)
    
    return df_styled

def ensure_numerical(df):
    return df.apply(pd.to_numeric, errors='coerce')

def plot_surface(nested_dict, udl1, udl2, date, surface_type, moneyness_levels, delta_levels, start_date=None, end_date=None, surface_data_type='Moneyness'):
    try:
        date = pd.Timestamp(date)
        levels = moneyness_levels if surface_data_type == 'Moneyness' else delta_levels

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl1, udl2, levels, surface_data_type)
            vol_surface = ensure_numerical(vol_surface)
            if vol_surface.isnull().all().all():
                print("The Volatility Surface contains only NaN values.")
                display(vol_surface.style.set_caption("Volatility Surface"))
            else:
                display(style_df(vol_surface, "Volatility Surface"))
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                percentile_surface = calculate_percentile_rank_surface(filtered_dict, udl1, udl2, end_date, levels, surface_data_type)
                percentile_surface = ensure_numerical(percentile_surface)
                title = f"Percentile Surface ({udl1} - {udl2}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(percentile_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                zscore_surface = calculate_zscore_surface(filtered_dict, udl1, udl2, end_date, levels, surface_data_type)
                zscore_surface = ensure_numerical(zscore_surface)
                title = f"Z-score Surface ({udl1} - {udl2}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(zscore_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Z-score surface.")
        else:
            print("Invalid surface type selected.")
    except KeyError as e:
        print(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        print(f"An error occurred: {e}")

def plot_surface_single(nested_dict, udl, date, surface_type, moneyness_levels, delta_levels, start_date=None, end_date=None, surface_data_type='Moneyness'):
    try:
        date = pd.Timestamp(date)
        levels = moneyness_levels if surface_data_type == 'Moneyness' else delta_levels

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl, None, levels, surface_data_type)
            vol_surface = ensure_numerical(vol_surface)
            if vol_surface.isnull().all().all():
                print("The Volatility Surface contains only NaN values.")
                display(vol_surface.style.set_caption("Volatility Surface"))
            else:
                display(style_df(vol_surface, "Volatility Surface"))
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                percentile_surface = calculate_percentile_rank_surface(nested_dict, udl, None, date, levels, start_date, end_date)
                percentile_surface = ensure_numerical(percentile_surface)
                title = f"Percentile Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(percentile_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                z_score_surface = calculate_zscore_surface(nested_dict, udl, None, date, levels, start_date, end_date)
                z_score_surface = ensure_numerical(z_score_surface)
                title = f"Z-score Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(z_score_surface, title)
                display(styled_df)
                print("Please select start and end dates for Z-score surface.")
        else:
            print("Invalid surface type selected.")
    except KeyError as e:
        print(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        print(f"An error occurred: {e}")

Data 

In [19]:
start_date = '2024-01-01'
end_date = '2024-05-28'
date = '2024-05-21'
udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P', 'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']
udl = 'EU_STOXX50E'
matu_list = [1, 3, 6, 9, 12, 18, 24, 36]
moneyness_list = [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
delta_list = [5, 10, 15, 20, 25, 35, 50, 65, 75, 90, 95]

path = "vol_surf.pickle"
with open(path, 'rb') as handle:
    nested_dict = pickle.load(handle)

### APP

In [21]:
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

# Function to toggle date widgets visibility
def toggle_date_widgets(surface_type):
    if surface_type in ['Percentile', 'Z-score']:
        start_date_widget.layout.display = 'block'
        end_date_widget.layout.display = 'block'
        date_widget.layout.display = 'none'
    else:
        start_date_widget.layout.display = 'none'
        end_date_widget.layout.display = 'none'
        date_widget.layout.display = 'block'

# Function to toggle between Moneyness and Delta widgets visibility
def toggle_surface_data_widgets(surface_data_type):
    if surface_data_type == 'Moneyness':
        moneyness_list_widget.layout.display = 'block'
        delta_list_widget.layout.display = 'none'
    else:
        moneyness_list_widget.layout.display = 'none'
        delta_list_widget.layout.display = 'block'

# Widget definitions
udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P', 'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']
udl_widget1 = widgets.Dropdown(options=udl_list, value='EU_STOXX50E', description='UDL 1:', disabled=False)
udl_widget2 = widgets.Dropdown(options=udl_list, value='JP_NKY', description='UDL 2:', disabled=False)
date_widget = widgets.DatePicker(description='Date', value=pd.to_datetime('2024-05-28'), disabled=False)
start_date_widget = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2024-01-01'), disabled=False)
end_date_widget = widgets.DatePicker(description='End Date', value=pd.to_datetime('2024-05-27'), disabled=False)
surface_type_widget = widgets.Dropdown(options=['Level', 'Percentile', 'Z-score'], value='Level', description='Type:', disabled=False)
surface_data_radio = widgets.RadioButtons(options=['Moneyness', 'Delta'], description='Surface Data:', disabled=False)

# Ensure the output widget is defined before using it in the interactive output
output = widgets.Output()

# Modified interactive output function
interactive_output = widgets.interactive_output(plot_surface, {
    'nested_dict': widgets.fixed(nested_dict),
    'udl1': udl_widget1,
    'udl2': udl_widget2,
    'date': date_widget,
    'surface_type': surface_type_widget,
    'moneyness_levels': widgets.fixed(moneyness_list),
    'delta_levels': widgets.fixed(delta_list),
    'start_date': start_date_widget,
    'end_date': end_date_widget,
    'surface_data_type': surface_data_radio
})

# Set up layout for widgets
left_box = widgets.VBox([udl_widget1, surface_type_widget], layout=widgets.Layout(margin='10px'))
right_box = widgets.VBox([start_date_widget, end_date_widget, date_widget], layout=widgets.Layout(margin='10px'))
top_box = widgets.HBox([left_box, right_box], layout=widgets.Layout(justify_content='space-between', align_items='center', margin='10px'))

spread_left_box = widgets.VBox([udl_widget1, udl_widget2, surface_type_widget], layout=widgets.Layout(margin='10px'))
spread_right_box = widgets.VBox([start_date_widget, end_date_widget, date_widget], layout=widgets.Layout(margin='10px'))
spread_top_box = widgets.HBox([spread_left_box, spread_right_box], layout=widgets.Layout(justify_content='space-between', align_items='center', margin='10px'))

moneyness_list_widget = widgets.Output()
delta_list_widget = widgets.Output()

# Initial widget setup
toggle_date_widgets(surface_type_widget.value)
toggle_surface_data_widgets(surface_data_radio.value)

# Observe changes in widgets to update the layout
surface_type_widget.observe(lambda change: toggle_date_widgets(change['new']), names='value')
surface_data_radio.observe(lambda change: toggle_surface_data_widgets(change['new']), names='value')

# Display the widgets and output
tab = widgets.Tab()
tab.children = [widgets.VBox([top_box, surface_data_radio, interactive_output]), widgets.VBox([spread_top_box, surface_data_radio, interactive_output])]
tab.set_title(0, 'Single UDL')
tab.set_title(1, 'Spread UDL')

display(tab)

# Merged plot_surface and plot_surface_single

In [7]:
# Corrected plot_surface function
def plot_surface(nested_dict, udl1, udl2, date, surface_type, moneyness_levels, delta_levels, start_date=None, end_date=None, surface_data_type='Moneyness'):
    try:
        date = pd.Timestamp(date)
        levels = moneyness_levels if surface_data_type == 'Moneyness' else delta_levels

        start_date_str = pd.Timestamp(start_date).strftime('%Y-%m-%d') if start_date else "N/A"
        end_date_str = pd.Timestamp(end_date).strftime('%Y-%m-%d') if end_date else "N/A"

        update_debug_info(f"Plotting surface for {udl1} and {udl2} on {date}, surface type: {surface_type}")
        update_debug_info(f"Start date: {start_date_str}, End date: {end_date_str}")

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl1, udl2, levels, surface_data_type)
            update_debug_info("Volatility Surface created successfully")
            vol_surface = ensure_numerical(vol_surface)
            if vol_surface.isnull().all().all():
                update_debug_info("The Volatility Surface contains only NaN values.")
                display(vol_surface.style.set_caption("Volatility Surface"))
            else:
                styled_df = style_df(vol_surface, "Volatility Surface")
                update_debug_info("Styled Volatility Surface DataFrame:")
                display(styled_df)
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    update_debug_info("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                update_debug_info(f"Filtered dictionary size: {len(filtered_dict)}")
                percentile_surface = calculate_percentile_rank_surface(filtered_dict, udl1, udl2, end_date, levels, surface_data_type)
                update_debug_info("Percentile Surface DataFrame:")
                update_debug_info(percentile_surface)
                percentile_surface = ensure_numerical(percentile_surface)
                title = f"Percentile Surface ({udl1} - {udl2}) From: {start_date_str} to: {end_date_str}"
                styled_df = style_df(percentile_surface, title)
                update_debug_info("Styled Percentile Surface DataFrame:")
                display(styled_df)
            else:
                update_debug_info("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    update_debug_info("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                update_debug_info(f"Filtered dictionary size: {len(filtered_dict)}")
                zscore_surface = calculate_zscore_surface(filtered_dict, udl1, udl2, end_date, levels, surface_data_type)
                update_debug_info("Z-score Surface DataFrame:")
                update_debug_info(zscore_surface)
                zscore_surface = ensure_numerical(zscore_surface)
                title = f"Z-score Surface ({udl1} - {udl2}) From: {start_date_str} to: {end_date_str}"
                styled_df = style_df(zscore_surface, title)
                update_debug_info("Styled Z-score Surface DataFrame:")
                display(styled_df)
            else:
                update_debug_info("Please select start and end dates for Z-score surface.")
        else:
            update_debug_info("Invalid surface type selected.")
    except KeyError as e:
        update_debug_info(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        update_debug_info(f"An error occurred: {e}")